# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:

from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('../../open_ai.env')) # read the env file from parent directory

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [4]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [5]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

The winner of the 2010 Formula 1 World Championship was Sebastian Vettel. He secured his first World Drivers' Championship driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [6]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

The winner of the 2011 Formula 1 World Championship was Sebastian Vettel. He drove for the Red Bull Racing team.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [7]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 Formula 1 World Championship was won by Fernando Alonso. He was driving for the Renault team.


In [8]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The winner of the 2019 Formula 1 World Championship was Lewis Hamilton.  
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [9]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

The winner of the 2019 Formula 1 World Championship was Lewis Hamilton.  
He drove for the Mercedes-AMG Petronas Formula One Team and secured his sixth World Drivers' Championship title that year.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [10]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton  
Team: Mercedes  
Points: 413


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The winner of the 2006 F1 championship was driver Fernando Alonso with the team Renault.


Few Shots for classification.


In [37]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [12]:
# Demonstration of additional message fields

# Example 1: Using 'name' field for multi-participant conversation
context_with_names = [
    {'role': 'system', 'content': 'You are facilitating a discussion between Alice and Bob about their favorite movies.'},
    {'role': 'user', 'content': 'I love action movies, especially Marvel films!', 'name': 'Alice'},
    {'role': 'user', 'content': 'I prefer romantic comedies myself.', 'name': 'Bob'},
    {'role': 'assistant', 'content': 'Interesting! Alice enjoys action-packed Marvel movies while Bob prefers romantic comedies. What specific movies do you each recommend?'},
    {'role': 'user', 'content': 'Definitely Avengers: Endgame!', 'name': 'Alice'},
]

print("=== Multi-participant conversation with 'name' field ===")
response = return_OAIResponse("What about you Bob, any specific romantic comedy recommendations?", context_with_names)
print(response)
print()

# Example 2: Demonstrating different content types
context_varied_content = [
    {'role': 'system', 'content': 'You are a helpful assistant that can handle various types of content.'},
    {'role': 'user', 'content': [
        {'type': 'text', 'text': 'What can you tell me about this?'},
        # Note: Image content would go here in vision models, but gpt-4-mini doesn't support images
    ]},
]

# Example 3: Showing how tool messages would work (conceptual, not executable without function setup)
context_with_tools_concept = [
    {'role': 'system', 'content': 'You are a weather assistant that can check current conditions.'},
    {'role': 'user', 'content': 'What\'s the weather in Paris?'},
    # This would be the assistant's response if it had tool calling enabled:
    {
        'role': 'assistant', 
        'content': None,
        'tool_calls': [
            {
                'id': 'call_abc123',
                'type': 'function',
                'function': {
                    'name': 'get_current_weather',
                    'arguments': '{"location": "Paris", "unit": "celsius"}'
                }
            }
        ]
    },
    # This would be the tool response:
    {
        'role': 'tool',
        'content': '{"location": "Paris", "temperature": "15", "condition": "partly cloudy"}',
        'tool_call_id': 'call_abc123'
    }
]

print("=== Explanation of other message fields ===")
print("1. 'name': Used to identify speakers in multi-participant conversations")
print("2. 'tool_calls': Used when assistant wants to call external functions")
print("3. 'tool_call_id': Links tool responses back to their requests") 
print("4. 'content': Can be string or array (for multimodal content in vision models)")
print()
print("Note: Tool calling requires specific API setup and function definitions.")

=== Multi-participant conversation with 'name' field ===
Absolutely! I highly recommend "Crazy Rich Asians." It's funny, heartwarming, and has a great cast. What about you, Alice? Any other Marvel movies besides Avengers: Endgame that you really like?

=== Explanation of other message fields ===
1. 'name': Used to identify speakers in multi-participant conversations
2. 'tool_calls': Used when assistant wants to call external functions
3. 'tool_call_id': Links tool responses back to their requests
4. 'content': Can be string or array (for multimodal content in vision models)

Note: Tool calling requires specific API setup and function definitions.
Absolutely! I highly recommend "Crazy Rich Asians." It's funny, heartwarming, and has a great cast. What about you, Alice? Any other Marvel movies besides Avengers: Endgame that you really like?

=== Explanation of other message fields ===
1. 'name': Used to identify speakers in multi-participant conversations
2. 'tool_calls': Used when assist

## Additional Message Fields in OpenAI Chat API

Besides `role` and `content`, there are several other optional fields you can include:

### Available Fields:
1. **`name`** - Identifies the speaker (useful for multi-participant conversations)
2. **`tool_calls`** - Array of function calls (for assistant messages)
3. **`tool_call_id`** - Links tool responses back to requests (for tool messages)

### Role-Specific Usage:
- **System messages**: Only `role` and `content`
- **User messages**: Can include `name` for identification
- **Assistant messages**: Can include `tool_calls` for function calling
- **Tool messages**: Must include `tool_call_id` to link responses

## Function Calling - Connecting to Actual Code

To connect OpenAI function calls to real code execution, you need:
1. Define your functions in Python
2. Create function schemas for OpenAI
3. Handle the function calling loop
4. Execute the actual functions and return results

In [ ]:
# Step 1: Define your actual Python functions
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather for a given location"""
    # This would normally call a real weather API
    # For demo purposes, we'll return mock data
    weather_data = {
        "New York": {"temperature": "72", "condition": "sunny", "humidity": "45%"},
        "London": {"temperature": "59", "condition": "cloudy", "humidity": "78%"},
        "Tokyo": {"temperature": "68", "condition": "rainy", "humidity": "85%"},
        "Paris": {"temperature": "64", "condition": "partly cloudy", "humidity": "52%"}
    }
    
    # Simple lookup with fallback
    weather = weather_data.get(location, {"temperature": "70", "condition": "unknown", "humidity": "50%"})
    
    if unit == "celsius":
        # Convert from fahrenheit to celsius
        temp_f = int(weather["temperature"])
        temp_c = round((temp_f - 32) * 5/9)
        weather["temperature"] = str(temp_c)
        weather["unit"] = "°C"
    else:
        weather["unit"] = "°F"
    
    return f"Weather in {location}: {weather['temperature']}{weather['unit']}, {weather['condition']}, humidity: {weather['humidity']}"

def calculate_math(expression):
    """Safely calculate a mathematical expression"""
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

# Test the functions
print("Testing functions:")
print(get_current_weather("New York"))
print(calculate_math("25 + 17 * 2"))

In [ ]:
# Step 2: Define function schemas for OpenAI
# These tell the model what functions are available and how to call them
function_definitions = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "Temperature unit"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "calculate_math",
            "description": "Calculate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate, e.g. '2 + 2' or '15 * 3'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

# Create a mapping from function names to actual functions
available_functions = {
    "get_current_weather": get_current_weather,
    "calculate_math": calculate_math
}

print("Function definitions ready!")

In [ ]:
# Step 3: Create a function to handle the complete function calling flow
def chat_with_functions(user_message, conversation_history=None):
    """Handle a conversation that can include function calls"""
    
    if conversation_history is None:
        conversation_history = [
            {"role": "system", "content": "You are a helpful assistant that can check weather and do math calculations."}
        ]
    
    # Add user message
    conversation_history.append({"role": "user", "content": user_message})
    
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    # Make initial request with function definitions
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Updated model name
        messages=conversation_history,
        tools=function_definitions,  # Pass available functions
        tool_choice="auto"  # Let model decide when to call functions
    )
    
    response_message = response.choices[0].message
    conversation_history.append(response_message)
    
    # Check if the model wants to call functions
    if response_message.tool_calls:
        print("🔧 Model wants to call functions:")
        
        # Execute each function call
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"  Calling {function_name} with args: {function_args}")
            
            # Call the actual function
            if function_name in available_functions:
                function_result = available_functions[function_name](**function_args)
                print(f"  Result: {function_result}")
                
                # Add function result to conversation
                conversation_history.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": function_result
                })
            else:
                print(f"  ❌ Unknown function: {function_name}")
        
        # Get final response from model after function calls
        final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversation_history
        )
        
        final_message = final_response.choices[0].message
        conversation_history.append(final_message)
        
        return final_message.content, conversation_history
    
    else:
        # No function calls needed
        return response_message.content, conversation_history

# Import json for parsing function arguments
import json

In [ ]:
# Step 4: Test the function calling system
print("=== Testing Function Calling System ===\n")

# Test 1: Weather function
print("Test 1: Weather query")
response, history = chat_with_functions("What's the weather like in New York?")
print(f"Final response: {response}")
print()

# Test 2: Math function  
print("Test 2: Math calculation")
response, history = chat_with_functions("What's 25 * 8 + 15?", conversation_history=None)
print(f"Final response: {response}")
print()

# Test 3: Multiple function calls in one request
print("Test 3: Multiple functions")
response, history = chat_with_functions("What's the weather in London and calculate 100 / 5 for me?")
print(f"Final response: {response}")
print()

# Test 4: No function needed
print("Test 4: Regular conversation")
response, history = chat_with_functions("Tell me a joke about programming")
print(f"Final response: {response}")

## How Function Calling Works - Step by Step

1. **Define Python Functions**: Write actual functions that do the work
2. **Create Function Schemas**: Tell OpenAI what functions exist and their parameters
3. **Make API Call**: Include `tools` parameter with function definitions
4. **Check Response**: See if model wants to call functions (`tool_calls`)
5. **Execute Functions**: Run the actual Python code
6. **Return Results**: Send function results back to model
7. **Get Final Response**: Model uses function results to answer user

### Key Points:
- The model decides when to call functions based on user input
- You execute the actual Python code, not OpenAI
- Function results are sent back to continue the conversation
- Multiple functions can be called in a single request